In [1]:
import pandas as pd
import numpy as np
import os
import pandas_gbq
import glob
from google.cloud import bigquery
import re
import openpyxl 

In [2]:
client = bigquery.Client()

In [3]:

query = ''' SELECT 

 cbo_2002,


  CASE 
    WHEN natureza_juridica IN ('1015','1040', '1074', '1104','1139', '1163', '1252', '1287', '1317', '1341') THEN 'Federal'
    WHEN natureza_juridica IN ('1023','1058', '1082', '1112', '1147', '1171', '1236','1260',  '1295', '1325') THEN 'Estadual'
    WHEN natureza_juridica IN ('1031','1066', '1120', '1155', '1180', '1244', '1279', '1309', '1333') THEN 'Municipal'
    ELSE 'Outros' 
  END AS esfera,

   CASE 
    WHEN sexo = '1' THEN 'Masculino'
    WHEN sexo = '2' THEN 'Feminino'
    ELSE 'Não identificado'
  END AS sexo,



  COUNT(*) AS quantidade_vinculos
  
  --avg(valor_remuneracao_media)as media


FROM `basedosdados.br_me_rais.microdados_vinculos`
WHERE (natureza_juridica LIKE "1%" 
OR natureza_juridica IN ('2011', '2038'))
AND natureza_juridica != '1228'
AND cbo_2002 NOT LIKE "0%" -- retirando os militares. 
AND vinculo_ativo_3112= '1' and ano=2022

GROUP BY 

cbo_2002,esfera,sexo

order by quantidade_vinculos desc
-- Total de 2105 codigos


--Quantidade de vinculos:
-- cbo, familia cbo, genero, esfera, poder

--Remuneração por ocupação:
-- cbo, familia cbo, genero, esfera, poder, tipo de adm (empresa pública devem ser as maiores)



 
 --Assistente administrativo
 --Professor de nível médio no ensino fundamental
 -- Professor de nível superior do ensino fundamental (primeira a quarta série)
 -- Dirigente do serviço público municipal
 -- Auxiliar de escritório
  '''

In [4]:
df = pandas_gbq.read_gbq(query, project_id='repositoriodedadosgpsp')
df


Downloading: 100%|██████████|


,cbo_2002,esfera,sexo,quantidade_vinculos
0,231210,Municipal,Feminino,324313
1,331205,Municipal,Feminino,313197
2,411010,Municipal,Feminino,273904
3,515105,Municipal,Feminino,187934
4,411010,Municipal,Masculino,185740
...,...,...,...,...
8889,842105,Federal,Masculino,1
8890,342515,Outros,Masculino,1
8891,374145,Estadual,Feminino,1
8892,142135,Federal,Feminino,1


In [6]:
df.sort_values(ascending=True,by='cbo_2002').head(30)

,cbo_2002,esfera,sexo,quantidade_vinculos
4662,111215,Federal,Masculino,13
6976,111215,Municipal,Feminino,3
7872,111215,Federal,Feminino,1
5959,111215,Municipal,Masculino,6
588,111220,Federal,Feminino,1545
181,111220,Municipal,Masculino,8360
2613,111220,Outros,Masculino,70
210,111220,Municipal,Feminino,6880
1454,111220,Estadual,Masculino,274
1105,111220,Federal,Masculino,482


In [7]:
df = df.rename(columns={'cbo_2002':'codigo'})
df

,codigo,esfera,sexo,quantidade_vinculos
0,231210,Municipal,Feminino,324313
1,331205,Municipal,Feminino,313197
2,411010,Municipal,Feminino,273904
3,515105,Municipal,Feminino,187934
4,411010,Municipal,Masculino,185740
...,...,...,...,...
8889,842105,Federal,Masculino,1
8890,342515,Outros,Masculino,1
8891,374145,Estadual,Feminino,1
8892,142135,Federal,Feminino,1


In [8]:
df.dtypes

codigo                 object
esfera                 object
sexo                   object
quantidade_vinculos     Int64
dtype: object

In [9]:
df['codigo'] = df['codigo'].astype(int)

Pegar dados cbo e dar merge com codigo do arquivo > CBO2002 - Ocupacao
subir pro datalake
desenhar gráfico


In [10]:
os.chdir('G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS E CONHECIMENTO\\415 - Repositório de Dados\\Atualização Dados novos\\CBO\\ESTRUTURA CBO')

In [11]:
os.listdir() 

['CBO2002 - Sinonimo.csv',
 'CBO2002 - PerfilOcupacional.csv',
 'CBO2002 - Ocupacao.csv',
 'CBO2002 - SubGrupo Principal.csv',
 'CBO2002 - Grande Grupo.csv',
 'CBO2002 - Familia.csv',
 'CBO2002 - SubGrupo.csv',
 'titulo_funcoes_cbo.xlsx',
 'CBO2002 - SubGrupo Principal.gsheet',
 'CBO2002 - SubGrupo.gsheet',
 'CBO2002 - Ocupação.xlsx',
 'comparando dados CBO.gsheet',
 'CBO2002 - Grande Grupo.gsheet',
 'cbo atualizado.xlsx',
 'cbo_2003.csv',
 'cbo_quadro.xlsx']

In [12]:
cbo = pd.read_excel("CBO2002 - Ocupação.xlsx")
cbo

,CODIGO,TITULO,Agrupamento
0,10105,Oficial general da aeronáutica,"MEMBROS DAS FORÇAS ARMADAS, POLICIAIS E BOMBEI..."
1,10110,Oficial general do exército,"MEMBROS DAS FORÇAS ARMADAS, POLICIAIS E BOMBEI..."
2,10115,Oficial general da marinha,"MEMBROS DAS FORÇAS ARMADAS, POLICIAIS E BOMBEI..."
3,10205,Oficial da aeronáutica,"MEMBROS DAS FORÇAS ARMADAS, POLICIAIS E BOMBEI..."
4,10210,Oficial do exército,"MEMBROS DAS FORÇAS ARMADAS, POLICIAIS E BOMBEI..."
...,...,...,...
2708,992205,Encarregado geral de operações de conservação ...,TRABALHADORES EM SERVIÇOS DE REPARAÇÃO E MANUT...
2709,992210,Encarregado de equipe de conservação de vias p...,TRABALHADORES EM SERVIÇOS DE REPARAÇÃO E MANUT...
2710,992215,Operador de ceifadeira na conservação de vias ...,TRABALHADORES EM SERVIÇOS DE REPARAÇÃO E MANUT...
2711,992220,Pedreiro de conservação de vias permanentes (e...,TRABALHADORES EM SERVIÇOS DE REPARAÇÃO E MANUT...


In [13]:
cbo['Agrupamento'].unique()

array(['MEMBROS DAS FORÇAS ARMADAS, POLICIAIS E BOMBEIROS MILITARES',
       'MEMBROS SUPERIORES DO PODER PÚBLICO, DIRIGENTES DE ORGANIZAÇÕES DE INTERESSE PÚBLICO E DE EMPRESAS, GERENTES',
       'PROFISSIONAIS DE MEDICINA, ENFERMAGEM E SERVIÇOS DA SAÚDE',
       'PROFISSIONAIS DE CIÊNCIA DE ENSINO SUPERIOR',
       'PROFISSIONAIS DE ENSINO',
       'PROFISSIONAIS DE ARTES DE ENSINO SUPERIOR',
       'TÉCNICOS DE NÍVEL MÉDIO',
       'TRABALHADORES DE SERVIÇOS ADMINISTRATIVOS',
       'TRABALHADORES DOS SERVIÇOS, VENDEDORES DO COMÉRCIO EM LOJAS E MERCADOS',
       'TRABALHADORES NOS SERVIÇOS DE PROTEÇÃO E SEGURANÇA',
       'TRABALHADORES AGROPECUÁRIOS, FLORESTAIS E DA PESCA',
       'TRABALHADORES DA PRODUÇÃO DE BENS E SERVIÇOS INDUSTRIAIS',
       'TRABALHADORES EM SERVIÇOS DE REPARAÇÃO E MANUTENÇÃO'],
      dtype=object)

In [14]:
cbo = cbo.rename(columns=
                 {
                     'CODIGO':'codigo',
                     'TITULO':'nome_ocupacao',
                     'Agrupamento':'grupo_ocupacao'
                 })
cbo

,codigo,nome_ocupacao,grupo_ocupacao
0,10105,Oficial general da aeronáutica,"MEMBROS DAS FORÇAS ARMADAS, POLICIAIS E BOMBEI..."
1,10110,Oficial general do exército,"MEMBROS DAS FORÇAS ARMADAS, POLICIAIS E BOMBEI..."
2,10115,Oficial general da marinha,"MEMBROS DAS FORÇAS ARMADAS, POLICIAIS E BOMBEI..."
3,10205,Oficial da aeronáutica,"MEMBROS DAS FORÇAS ARMADAS, POLICIAIS E BOMBEI..."
4,10210,Oficial do exército,"MEMBROS DAS FORÇAS ARMADAS, POLICIAIS E BOMBEI..."
...,...,...,...
2708,992205,Encarregado geral de operações de conservação ...,TRABALHADORES EM SERVIÇOS DE REPARAÇÃO E MANUT...
2709,992210,Encarregado de equipe de conservação de vias p...,TRABALHADORES EM SERVIÇOS DE REPARAÇÃO E MANUT...
2710,992215,Operador de ceifadeira na conservação de vias ...,TRABALHADORES EM SERVIÇOS DE REPARAÇÃO E MANUT...
2711,992220,Pedreiro de conservação de vias permanentes (e...,TRABALHADORES EM SERVIÇOS DE REPARAÇÃO E MANUT...


In [16]:
df[df['codigo']==517335]  #Guarda portuário cod cbo novo

,codigo,esfera,sexo,quantidade_vinculos
1949,517335,Outros,Masculino,140
6282,517335,Outros,Feminino,4


In [17]:
df[df['codigo']==517235] #Guarda portuário codigo cbo velho

,codigo,esfera,sexo,quantidade_vinculos
838,517235,Outros,Masculino,799
2409,517235,Outros,Feminino,86
2857,517235,Estadual,Masculino,58
3478,517235,Municipal,Masculino,34
6831,517235,Estadual,Feminino,3
7395,517235,Municipal,Feminino,2


In [18]:
cbo[cbo['codigo']==517235]

,codigo,nome_ocupacao,grupo_ocupacao
1513,517235,Guarda portuário,TRABALHADORES NOS SERVIÇOS DE PROTEÇÃO E SEGUR...


In [19]:
cbo['grupo_ocupacao'] = cbo['grupo_ocupacao'].str.title()
cbo

,codigo,nome_ocupacao,grupo_ocupacao
0,10105,Oficial general da aeronáutica,"Membros Das Forças Armadas, Policiais E Bombei..."
1,10110,Oficial general do exército,"Membros Das Forças Armadas, Policiais E Bombei..."
2,10115,Oficial general da marinha,"Membros Das Forças Armadas, Policiais E Bombei..."
3,10205,Oficial da aeronáutica,"Membros Das Forças Armadas, Policiais E Bombei..."
4,10210,Oficial do exército,"Membros Das Forças Armadas, Policiais E Bombei..."
...,...,...,...
2708,992205,Encarregado geral de operações de conservação ...,Trabalhadores Em Serviços De Reparação E Manut...
2709,992210,Encarregado de equipe de conservação de vias p...,Trabalhadores Em Serviços De Reparação E Manut...
2710,992215,Operador de ceifadeira na conservação de vias ...,Trabalhadores Em Serviços De Reparação E Manut...
2711,992220,Pedreiro de conservação de vias permanentes (e...,Trabalhadores Em Serviços De Reparação E Manut...


In [20]:
cbo_merge = df.merge(cbo, how='left', on='codigo')
cbo_merge 

,codigo,esfera,sexo,quantidade_vinculos,nome_ocupacao,grupo_ocupacao
0,231210,Municipal,Feminino,324313,Professor de nível superior do ensino fundamen...,Profissionais De Ensino
1,331205,Municipal,Feminino,313197,Professor de nível médio no ensino fundamental,Profissionais De Ensino
2,411010,Municipal,Feminino,273904,Assistente administrativo,Trabalhadores De Serviços Administrativos
3,515105,Municipal,Feminino,187934,Agente comunitário de saúde,"Profissionais De Medicina, Enfermagem E Serviç..."
4,411010,Municipal,Masculino,185740,Assistente administrativo,Trabalhadores De Serviços Administrativos
...,...,...,...,...,...,...
8889,842105,Federal,Masculino,1,Preparador de melado e essência de fumo,Trabalhadores Da Produção De Bens E Serviços I...
8890,342515,Outros,Masculino,1,Fiscal de aviação civil (fac),Técnicos De Nível Médio
8891,374145,Estadual,Feminino,1,Dj (disc jockey),Técnicos De Nível Médio
8892,142135,Federal,Feminino,1,Oficial de proteção de dados pessoais (dpo),"Membros Superiores Do Poder Público, Dirigente..."


In [21]:
cbo_merge.dtypes

codigo                  int32
esfera                 object
sexo                   object
quantidade_vinculos     Int64
nome_ocupacao          object
grupo_ocupacao         object
dtype: object

In [22]:
cbo_merge['grupo_ocupacao'].unique()

array(['Profissionais De Ensino',
       'Trabalhadores De Serviços Administrativos',
       'Profissionais De Medicina, Enfermagem E Serviços Da Saúde',
       'Membros Superiores Do Poder Público, Dirigentes De Organizações De Interesse Público E De Empresas, Gerentes',
       'Trabalhadores Dos Serviços, Vendedores Do Comércio Em Lojas E Mercados',
       'Trabalhadores Nos Serviços De Proteção E Segurança',
       'Trabalhadores Da Produção De Bens E Serviços Industriais',
       'Técnicos De Nível Médio',
       'Trabalhadores Em Serviços De Reparação E Manutenção',
       'Profissionais De Ciência De Ensino Superior',
       'Trabalhadores Agropecuários, Florestais E Da Pesca', nan,
       'Profissionais De Artes De Ensino Superior'], dtype=object)

In [23]:
len(cbo_merge['grupo_ocupacao'].unique())

13

In [24]:
cod_obsoleto = cbo_merge[cbo_merge['nome_ocupacao'].isna()] #codigos excluidos e desatualizados
cod_obsoleto 

,codigo,esfera,sexo,quantidade_vinculos,nome_ocupacao,grupo_ocupacao
417,999999,Outros,Masculino,2593,NaN,NaN
705,999999,Outros,Feminino,1147,NaN,NaN
1119,999999,Municipal,Feminino,469,NaN,NaN
1434,373205,Outros,Masculino,283,NaN,NaN
1919,999999,Federal,Masculino,147,NaN,NaN
1949,517335,Outros,Masculino,140,NaN,NaN
2095,999999,Municipal,Masculino,119,NaN,NaN
2221,999999,Federal,Feminino,105,NaN,NaN
2418,322105,Municipal,Feminino,85,NaN,NaN
2455,373205,Outros,Feminino,83,NaN,NaN


In [25]:
len(cod_obsoleto)

30

In [26]:
cod_obsoleto['codigo'].unique()

array([999999, 373205, 517335, 322105, 322120, 373115, 373210, 322125,
       373215, 322115, 722225])

In [27]:
adicionar_nome = {
    999999: {'nome_ocupacao': 'Desconhecido Ou Não Informado', 'grupo_ocupacao': 'Trabalhadores Em Serviços De Reparação E Manutenção'},
    373205: {'nome_ocupacao': 'Técnico De Sistemas Audiovisuais', 'grupo_ocupacao': 'Técnicos De Nível Médio'}, 
    517335: {'nome_ocupacao': 'Guarda Portuário', 'grupo_ocupacao': 'Trabalhadores Nos Serviços De Proteção E Segurança'},
    322105: {'nome_ocupacao': 'Técnico Em Acupuntura', 'grupo_ocupacao': 'Técnicos De Nível Médio'},
    322120: {'nome_ocupacao': 'Massoterapeuta', 'grupo_ocupacao': 'Técnicos De Nível Médio'},
    373115: {'nome_ocupacao': 'Operador De Externa (Rádio)', 'grupo_ocupacao': 'Técnicos De Nível Médio'},
    373210: {'nome_ocupacao': 'Operador De Controle Mestre', 'grupo_ocupacao': 'Técnicos De Nível Médio'},
    322125: {'nome_ocupacao': 'Terapeuta Holístico', 'grupo_ocupacao': 'Técnicos De Nível Médio'},
    373215: {'nome_ocupacao': 'Técnico Em Operação De Equipamentos De Transmissão/Recepção De Televisão', 'grupo_ocupacao': 'Técnicos De Nível Médio'},
    322115: {'nome_ocupacao': 'Técnico Em Quiropraxia', 'grupo_ocupacao': 'Técnicos De Nível Médio'},
    722225: {'nome_ocupacao': 'Operador De Máquina De Fundir Sob Pressão', 'grupo_ocupacao': 'Trabalhadores Da Produção De Bens E Serviços Industriais'}
} # Reagrupando de acordo com CBO no MTE


In [28]:
def adicionar(row):
    if pd.isna(row['nome_ocupacao']) and row['codigo'] in adicionar_nome:
        return pd.Series({
            'nome_ocupacao': adicionar_nome[row['codigo']]['nome_ocupacao'],
            'grupo_ocupacao': adicionar_nome[row['codigo']]['grupo_ocupacao']
        })
    return pd.Series({'nome_ocupacao': row['nome_ocupacao'], 'grupo_ocupacao': row['grupo_ocupacao']})

In [29]:
cbo_merge[['nome_ocupacao', 'grupo_ocupacao']] = cbo_merge.apply(adicionar, axis=1)

In [30]:
cbo_merge[cbo_merge['nome_ocupacao']=='Técnico de sistemas audiovisuais']

,codigo,esfera,sexo,quantidade_vinculos,nome_ocupacao,grupo_ocupacao
1079,373130,Federal,Masculino,508,Técnico de sistemas audiovisuais,Técnicos De Nível Médio
3068,373130,Federal,Feminino,47,Técnico de sistemas audiovisuais,Técnicos De Nível Médio
3621,373130,Estadual,Masculino,30,Técnico de sistemas audiovisuais,Técnicos De Nível Médio
4825,373130,Municipal,Masculino,12,Técnico de sistemas audiovisuais,Técnicos De Nível Médio
5660,373130,Outros,Masculino,7,Técnico de sistemas audiovisuais,Técnicos De Nível Médio
7351,373130,Estadual,Feminino,2,Técnico de sistemas audiovisuais,Técnicos De Nível Médio
8069,373130,Outros,Feminino,1,Técnico de sistemas audiovisuais,Técnicos De Nível Médio


In [31]:
cbo_merge[cbo_merge['nome_ocupacao']=='Guarda Portuário']

,codigo,esfera,sexo,quantidade_vinculos,nome_ocupacao,grupo_ocupacao
1949,517335,Outros,Masculino,140,Guarda Portuário,Trabalhadores Nos Serviços De Proteção E Segur...
6282,517335,Outros,Feminino,4,Guarda Portuário,Trabalhadores Nos Serviços De Proteção E Segur...


atualizado codigo

In [32]:
atualizar_codigo = {
    373205: 373130,
    517335: 517235,
    322105: 322705,
    322120: 322715,
    373210: 373135,
    322125: 322720,
    322115: 322710
}

In [33]:
cbo_merge['codigo'] = cbo_merge['codigo'].replace(atualizar_codigo)


In [34]:
cbo_merge

,codigo,esfera,sexo,quantidade_vinculos,nome_ocupacao,grupo_ocupacao
0,231210,Municipal,Feminino,324313,Professor de nível superior do ensino fundamen...,Profissionais De Ensino
1,331205,Municipal,Feminino,313197,Professor de nível médio no ensino fundamental,Profissionais De Ensino
2,411010,Municipal,Feminino,273904,Assistente administrativo,Trabalhadores De Serviços Administrativos
3,515105,Municipal,Feminino,187934,Agente comunitário de saúde,"Profissionais De Medicina, Enfermagem E Serviç..."
4,411010,Municipal,Masculino,185740,Assistente administrativo,Trabalhadores De Serviços Administrativos
...,...,...,...,...,...,...
8889,842105,Federal,Masculino,1,Preparador de melado e essência de fumo,Trabalhadores Da Produção De Bens E Serviços I...
8890,342515,Outros,Masculino,1,Fiscal de aviação civil (fac),Técnicos De Nível Médio
8891,374145,Estadual,Feminino,1,Dj (disc jockey),Técnicos De Nível Médio
8892,142135,Federal,Feminino,1,Oficial de proteção de dados pessoais (dpo),"Membros Superiores Do Poder Público, Dirigente..."


In [35]:
cbo_merge['nome_ocupacao'] = cbo_merge['nome_ocupacao'].str.title()
cbo_merge

,codigo,esfera,sexo,quantidade_vinculos,nome_ocupacao,grupo_ocupacao
0,231210,Municipal,Feminino,324313,Professor De Nível Superior Do Ensino Fundamen...,Profissionais De Ensino
1,331205,Municipal,Feminino,313197,Professor De Nível Médio No Ensino Fundamental,Profissionais De Ensino
2,411010,Municipal,Feminino,273904,Assistente Administrativo,Trabalhadores De Serviços Administrativos
3,515105,Municipal,Feminino,187934,Agente Comunitário De Saúde,"Profissionais De Medicina, Enfermagem E Serviç..."
4,411010,Municipal,Masculino,185740,Assistente Administrativo,Trabalhadores De Serviços Administrativos
...,...,...,...,...,...,...
8889,842105,Federal,Masculino,1,Preparador De Melado E Essência De Fumo,Trabalhadores Da Produção De Bens E Serviços I...
8890,342515,Outros,Masculino,1,Fiscal De Aviação Civil (Fac),Técnicos De Nível Médio
8891,374145,Estadual,Feminino,1,Dj (Disc Jockey),Técnicos De Nível Médio
8892,142135,Federal,Feminino,1,Oficial De Proteção De Dados Pessoais (Dpo),"Membros Superiores Do Poder Público, Dirigente..."


In [36]:
len(cbo_merge['grupo_ocupacao'].unique())

12

In [37]:
cbo_merge['grupo_ocupacao'].unique()

array(['Profissionais De Ensino',
       'Trabalhadores De Serviços Administrativos',
       'Profissionais De Medicina, Enfermagem E Serviços Da Saúde',
       'Membros Superiores Do Poder Público, Dirigentes De Organizações De Interesse Público E De Empresas, Gerentes',
       'Trabalhadores Dos Serviços, Vendedores Do Comércio Em Lojas E Mercados',
       'Trabalhadores Nos Serviços De Proteção E Segurança',
       'Trabalhadores Da Produção De Bens E Serviços Industriais',
       'Técnicos De Nível Médio',
       'Trabalhadores Em Serviços De Reparação E Manutenção',
       'Profissionais De Ciência De Ensino Superior',
       'Trabalhadores Agropecuários, Florestais E Da Pesca',
       'Profissionais De Artes De Ensino Superior'], dtype=object)

In [38]:
cbo_merge['grupo_ocupacao'] = cbo_merge['grupo_ocupacao'].str.title()
cbo_merge

,codigo,esfera,sexo,quantidade_vinculos,nome_ocupacao,grupo_ocupacao
0,231210,Municipal,Feminino,324313,Professor De Nível Superior Do Ensino Fundamen...,Profissionais De Ensino
1,331205,Municipal,Feminino,313197,Professor De Nível Médio No Ensino Fundamental,Profissionais De Ensino
2,411010,Municipal,Feminino,273904,Assistente Administrativo,Trabalhadores De Serviços Administrativos
3,515105,Municipal,Feminino,187934,Agente Comunitário De Saúde,"Profissionais De Medicina, Enfermagem E Serviç..."
4,411010,Municipal,Masculino,185740,Assistente Administrativo,Trabalhadores De Serviços Administrativos
...,...,...,...,...,...,...
8889,842105,Federal,Masculino,1,Preparador De Melado E Essência De Fumo,Trabalhadores Da Produção De Bens E Serviços I...
8890,342515,Outros,Masculino,1,Fiscal De Aviação Civil (Fac),Técnicos De Nível Médio
8891,374145,Estadual,Feminino,1,Dj (Disc Jockey),Técnicos De Nível Médio
8892,142135,Federal,Feminino,1,Oficial De Proteção De Dados Pessoais (Dpo),"Membros Superiores Do Poder Público, Dirigente..."


In [39]:
cbo_merge[cbo_merge['nome_ocupacao']=='Guarda Portuário'] #Tudo padronizado. é preciso somar os vinculos

,codigo,esfera,sexo,quantidade_vinculos,nome_ocupacao,grupo_ocupacao
838,517235,Outros,Masculino,799,Guarda Portuário,Trabalhadores Nos Serviços De Proteção E Segur...
1949,517235,Outros,Masculino,140,Guarda Portuário,Trabalhadores Nos Serviços De Proteção E Segur...
2409,517235,Outros,Feminino,86,Guarda Portuário,Trabalhadores Nos Serviços De Proteção E Segur...
2857,517235,Estadual,Masculino,58,Guarda Portuário,Trabalhadores Nos Serviços De Proteção E Segur...
3478,517235,Municipal,Masculino,34,Guarda Portuário,Trabalhadores Nos Serviços De Proteção E Segur...
6282,517235,Outros,Feminino,4,Guarda Portuário,Trabalhadores Nos Serviços De Proteção E Segur...
6831,517235,Estadual,Feminino,3,Guarda Portuário,Trabalhadores Nos Serviços De Proteção E Segur...
7395,517235,Municipal,Feminino,2,Guarda Portuário,Trabalhadores Nos Serviços De Proteção E Segur...


In [40]:
cbo_merge['grupo_ocupacao'].unique()

array(['Profissionais De Ensino',
       'Trabalhadores De Serviços Administrativos',
       'Profissionais De Medicina, Enfermagem E Serviços Da Saúde',
       'Membros Superiores Do Poder Público, Dirigentes De Organizações De Interesse Público E De Empresas, Gerentes',
       'Trabalhadores Dos Serviços, Vendedores Do Comércio Em Lojas E Mercados',
       'Trabalhadores Nos Serviços De Proteção E Segurança',
       'Trabalhadores Da Produção De Bens E Serviços Industriais',
       'Técnicos De Nível Médio',
       'Trabalhadores Em Serviços De Reparação E Manutenção',
       'Profissionais De Ciência De Ensino Superior',
       'Trabalhadores Agropecuários, Florestais E Da Pesca',
       'Profissionais De Artes De Ensino Superior'], dtype=object)

In [41]:
cbo_merge = cbo_merge.groupby(['codigo', 'esfera', 'sexo', 'nome_ocupacao', 'grupo_ocupacao'], as_index=False).agg({'quantidade_vinculos': 'sum'})
cbo_merge

,codigo,esfera,sexo,nome_ocupacao,grupo_ocupacao,quantidade_vinculos
0,111215,Federal,Feminino,Ministro De Estado,"Membros Superiores Do Poder Público, Dirigente...",1
1,111215,Federal,Masculino,Ministro De Estado,"Membros Superiores Do Poder Público, Dirigente...",13
2,111215,Municipal,Feminino,Ministro De Estado,"Membros Superiores Do Poder Público, Dirigente...",3
3,111215,Municipal,Masculino,Ministro De Estado,"Membros Superiores Do Poder Público, Dirigente...",6
4,111220,Estadual,Feminino,Secretário - Executivo,"Membros Superiores Do Poder Público, Dirigente...",304
...,...,...,...,...,...,...
8884,999999,Federal,Masculino,Desconhecido Ou Não Informado,Trabalhadores Em Serviços De Reparação E Manut...,147
8885,999999,Municipal,Feminino,Desconhecido Ou Não Informado,Trabalhadores Em Serviços De Reparação E Manut...,469
8886,999999,Municipal,Masculino,Desconhecido Ou Não Informado,Trabalhadores Em Serviços De Reparação E Manut...,119
8887,999999,Outros,Feminino,Desconhecido Ou Não Informado,Trabalhadores Em Serviços De Reparação E Manut...,1147


In [42]:
cbo_merge[cbo_merge['nome_ocupacao']=='Guarda Portuário'] # Vinculos somados 

,codigo,esfera,sexo,nome_ocupacao,grupo_ocupacao,quantidade_vinculos
6768,517235,Estadual,Feminino,Guarda Portuário,Trabalhadores Nos Serviços De Proteção E Segur...,3
6769,517235,Estadual,Masculino,Guarda Portuário,Trabalhadores Nos Serviços De Proteção E Segur...,58
6770,517235,Municipal,Feminino,Guarda Portuário,Trabalhadores Nos Serviços De Proteção E Segur...,2
6771,517235,Municipal,Masculino,Guarda Portuário,Trabalhadores Nos Serviços De Proteção E Segur...,34
6772,517235,Outros,Feminino,Guarda Portuário,Trabalhadores Nos Serviços De Proteção E Segur...,90
6773,517235,Outros,Masculino,Guarda Portuário,Trabalhadores Nos Serviços De Proteção E Segur...,939


In [43]:
cbo_merge['ano'] = 2022 # Adicionar ano

In [44]:
cbo_merge = cbo_merge.rename(columns={'sexo':'genero'})


In [45]:
cbo_merge = cbo_merge[['ano','codigo','nome_ocupacao','grupo_ocupacao','esfera','genero','quantidade_vinculos']]

In [46]:
cbo_merge.dtypes

ano                     int64
codigo                  int32
nome_ocupacao          object
grupo_ocupacao         object
esfera                 object
genero                 object
quantidade_vinculos     Int64
dtype: object

In [48]:
cbo_merge['grupo_ocupacao'].unique()

array(['Membros Superiores Do Poder Público, Dirigentes De Organizações De Interesse Público E De Empresas, Gerentes',
       'Profissionais De Medicina, Enfermagem E Serviços Da Saúde',
       'Profissionais De Ciência De Ensino Superior',
       'Profissionais De Ensino',
       'Profissionais De Artes De Ensino Superior',
       'Técnicos De Nível Médio',
       'Trabalhadores De Serviços Administrativos',
       'Trabalhadores Dos Serviços, Vendedores Do Comércio Em Lojas E Mercados',
       'Trabalhadores Nos Serviços De Proteção E Segurança',
       'Trabalhadores Agropecuários, Florestais E Da Pesca',
       'Trabalhadores Da Produção De Bens E Serviços Industriais',
       'Trabalhadores Em Serviços De Reparação E Manutenção'],
      dtype=object)

In [49]:
dataset_ref = client.dataset('perfil_remuneracao')

In [50]:
schema = [bigquery.SchemaField('ano', 'INTEGER', description= 'Ano de referência da observação'),
          bigquery.SchemaField('codigo', 'INTEGER', description= 'Código da Classificação Brasileira de Ocupações (CBO)'),
          bigquery.SchemaField('nome_ocupacao', 'STRING', description= 'Nome da ocupação na CBO'),
          bigquery.SchemaField('grupo_ocupacao', 'STRING', description= 'Agrupamento da ocupação'),
          bigquery.SchemaField('esfera', 'STRING', description= 'Nível da esfera do governo referente da observação'),                    
          bigquery.SchemaField('genero', 'STRING', description= 'Gênero autodeclarado ou não'),                    
          bigquery.SchemaField('quantidade_vinculos', 'INTEGER', description= 'Número total de vinculos observados')
]

In [51]:
table_ref = dataset_ref.table('RAIS_cbo_2022_quantidade_vinculos')
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(cbo_merge, table_ref, job_config=job_config)
job.result() 

LoadJob<project=repositoriodedadosgpsp, location=US, id=85bd07a6-6401-4007-b69d-a99389216510>

analise

In [3]:
query = '''SELECT *  FROM `repositoriodedadosgpsp.perfil_remuneracao.RAIS_cbo_2022_quantidade_vinculos`'''

In [4]:
df = pandas_gbq.read_gbq(query, project_id='repositoriodedadosgpsp')
df


Downloading: 100%|██████████|


,ano,codigo,nome_ocupacao,grupo_ocupacao,esfera,genero,quantidade_vinculos
0,2022,231105,Professor De Nível Superior Na Educação Infant...,Profissionais De Ensino,Estadual,Feminino,113
1,2022,231105,Professor De Nível Superior Na Educação Infant...,Profissionais De Ensino,Federal,Feminino,246
2,2022,231105,Professor De Nível Superior Na Educação Infant...,Profissionais De Ensino,Municipal,Feminino,81881
3,2022,231105,Professor De Nível Superior Na Educação Infant...,Profissionais De Ensino,Outros,Feminino,2
4,2022,231110,Professor De Nível Superior Na Educação Infant...,Profissionais De Ensino,Estadual,Feminino,12
...,...,...,...,...,...,...,...
8884,2022,142705,Gerente De Projetos E Serviços De Manutenção,"Membros Superiores Do Poder Público, Dirigente...",Municipal,Masculino,709
8885,2022,142705,Gerente De Projetos E Serviços De Manutenção,"Membros Superiores Do Poder Público, Dirigente...",Outros,Masculino,231
8886,2022,142710,Tecnólogo Em Sistemas Biomédicos,"Membros Superiores Do Poder Público, Dirigente...",Estadual,Masculino,32
8887,2022,142710,Tecnólogo Em Sistemas Biomédicos,"Membros Superiores Do Poder Público, Dirigente...",Federal,Masculino,576


In [58]:
df.columns

Index(['ano', 'codigo', 'nome_ocupacao', 'grupo_ocupacao', 'esfera', 'genero',
       'quantidade_vinculos'],
      dtype='object')

In [59]:
df['grupo_ocupacao'].unique()

array(['Profissionais De Ensino', 'Técnicos De Nível Médio',
       'Profissionais De Artes De Ensino Superior',
       'Trabalhadores De Serviços Administrativos',
       'Profissionais De Ciência De Ensino Superior',
       'Trabalhadores Agropecuários, Florestais E Da Pesca',
       'Trabalhadores Nos Serviços De Proteção E Segurança',
       'Trabalhadores Em Serviços De Reparação E Manutenção',
       'Profissionais De Medicina, Enfermagem E Serviços Da Saúde',
       'Trabalhadores Da Produção De Bens E Serviços Industriais',
       'Trabalhadores Dos Serviços, Vendedores Do Comércio Em Lojas E Mercados',
       'Membros Superiores Do Poder Público, Dirigentes De Organizações De Interesse Público E De Empresas, Gerentes'],
      dtype=object)

In [49]:
cat9 = df[df['grupo_ocupacao']=="Trabalhadores Dos Serviços, Vendedores Do Comércio Em Lojas E Mercados"]

In [50]:
cat9

,ano,codigo,nome_ocupacao,grupo_ocupacao,esfera,genero,quantidade_vinculos
7469,2022,510105,Supervisor De Transportes,"Trabalhadores Dos Serviços, Vendedores Do Comé...",Estadual,Feminino,47
7470,2022,510105,Supervisor De Transportes,"Trabalhadores Dos Serviços, Vendedores Do Comé...",Federal,Feminino,1
7471,2022,510105,Supervisor De Transportes,"Trabalhadores Dos Serviços, Vendedores Do Comé...",Municipal,Feminino,226
7472,2022,510105,Supervisor De Transportes,"Trabalhadores Dos Serviços, Vendedores Do Comé...",Outros,Feminino,12
7473,2022,510110,Administrador De Edifícios,"Trabalhadores Dos Serviços, Vendedores Do Comé...",Estadual,Feminino,83
...,...,...,...,...,...,...,...
8179,2022,523115,Chaveiro,"Trabalhadores Dos Serviços, Vendedores Do Comé...",Municipal,Masculino,2
8180,2022,524205,Feirante,"Trabalhadores Dos Serviços, Vendedores Do Comé...",Municipal,Masculino,33
8181,2022,524215,Vendedor Permissionário,"Trabalhadores Dos Serviços, Vendedores Do Comé...",Municipal,Masculino,6
8182,2022,524305,Vendedor Ambulante,"Trabalhadores Dos Serviços, Vendedores Do Comé...",Federal,Masculino,2


In [ ]:
cat9['quantidade_vinculos'].sum()

In [ ]:
cat9 = cat9.groupby(['codigo','nome_ocupacao'], as_index=False).agg({'quantidade_vinculos': 'sum'}) 


In [ ]:
cat9.sort_values(ascending=False, by='quantidade_vinculos').head(20)

# analises

In [5]:
os.chdir('G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS E CONHECIMENTO\\415 - Repositório de Dados\\Atualização Dados novos\\CBO\\Analise CBO')

In [6]:
os.listdir()

['errado.gsheet',
 'total por grupo 2022.xlsx',
 'dados comparativos - CBO 2003 2022 .xlsx',
 'total por grupo 2003.xlsx',
 'top10 profissoes dos Trabalhadores De Serviços Administrativos 2003.csv',
 '2003 top 10 profissoes + %.xlsx']

In [8]:
df[df['nome_ocupacao']=='Auxiliar De Enfermagem']

,ano,codigo,nome_ocupacao,grupo_ocupacao,esfera,genero,quantidade_vinculos
5619,2022,322230,Auxiliar De Enfermagem,"Profissionais De Medicina, Enfermagem E Serviç...",Estadual,Feminino,34517
5620,2022,322230,Auxiliar De Enfermagem,"Profissionais De Medicina, Enfermagem E Serviç...",Federal,Feminino,8517
5621,2022,322230,Auxiliar De Enfermagem,"Profissionais De Medicina, Enfermagem E Serviç...",Municipal,Feminino,63337
5622,2022,322230,Auxiliar De Enfermagem,"Profissionais De Medicina, Enfermagem E Serviç...",Outros,Feminino,1201
6158,2022,322230,Auxiliar De Enfermagem,"Profissionais De Medicina, Enfermagem E Serviç...",Estadual,Masculino,6115
6159,2022,322230,Auxiliar De Enfermagem,"Profissionais De Medicina, Enfermagem E Serviç...",Federal,Masculino,1589
6160,2022,322230,Auxiliar De Enfermagem,"Profissionais De Medicina, Enfermagem E Serviç...",Municipal,Masculino,9818
6161,2022,322230,Auxiliar De Enfermagem,"Profissionais De Medicina, Enfermagem E Serviç...",Outros,Masculino,222


In [9]:
df_agrupado = df.groupby('grupo_ocupacao')['quantidade_vinculos'].sum().reset_index()
df_agrupado 

,grupo_ocupacao,quantidade_vinculos
0,"Membros Superiores Do Poder Público, Dirigente...",915395
1,Profissionais De Artes De Ensino Superior,5681
2,Profissionais De Ciência De Ensino Superior,537624
3,Profissionais De Ensino,2670616
4,"Profissionais De Medicina, Enfermagem E Serviç...",1425713
5,"Trabalhadores Agropecuários, Florestais E Da P...",20574
6,Trabalhadores Da Produção De Bens E Serviços I...,403760
7,Trabalhadores De Serviços Administrativos,1862521
8,"Trabalhadores Dos Serviços, Vendedores Do Comé...",917772
9,Trabalhadores Em Serviços De Reparação E Manut...,77284


In [10]:
df['quantidade_vinculos'].sum() 

9966404

In [11]:
df_agrupado['grupo_ocupacao'].unique()

array(['Membros Superiores Do Poder Público, Dirigentes De Organizações De Interesse Público E De Empresas, Gerentes',
       'Profissionais De Artes De Ensino Superior',
       'Profissionais De Ciência De Ensino Superior',
       'Profissionais De Ensino',
       'Profissionais De Medicina, Enfermagem E Serviços Da Saúde',
       'Trabalhadores Agropecuários, Florestais E Da Pesca',
       'Trabalhadores Da Produção De Bens E Serviços Industriais',
       'Trabalhadores De Serviços Administrativos',
       'Trabalhadores Dos Serviços, Vendedores Do Comércio Em Lojas E Mercados',
       'Trabalhadores Em Serviços De Reparação E Manutenção',
       'Trabalhadores Nos Serviços De Proteção E Segurança',
       'Técnicos De Nível Médio'], dtype=object)

In [12]:
total = df_agrupado['quantidade_vinculos'].sum()

In [14]:
df_agrupado['proporcao'] = (df_agrupado['quantidade_vinculos'] / total) * 100

In [17]:
df_agrupado.sort_values(ascending=False, by='proporcao').head(10)

,grupo_ocupacao,quantidade_vinculos,proporcao
3,Profissionais De Ensino,2670616,26.796184
7,Trabalhadores De Serviços Administrativos,1862521,18.687994
4,"Profissionais De Medicina, Enfermagem E Serviç...",1425713,14.30519
8,"Trabalhadores Dos Serviços, Vendedores Do Comé...",917772,9.208657
0,"Membros Superiores Do Poder Público, Dirigente...",915395,9.184807
11,Técnicos De Nível Médio,711138,7.135352
2,Profissionais De Ciência De Ensino Superior,537624,5.394363
10,Trabalhadores Nos Serviços De Proteção E Segur...,418326,4.197361
6,Trabalhadores Da Produção De Bens E Serviços I...,403760,4.05121
9,Trabalhadores Em Serviços De Reparação E Manut...,77284,0.775445


# Por profissão

pegar
1. Profissionais De Ensino
2. Trabalhadores De Serviços Administrativos
3. Profissionais De Medicina, Enfermagem E Serviços Da Saúde




In [18]:
agrup = df.groupby(['nome_ocupacao', 'grupo_ocupacao']).agg({'quantidade_vinculos': 'sum'}).reset_index() #top profissões
top10 = agrup.sort_values(ascending=False, by='quantidade_vinculos').head(10)

In [19]:
top10

,nome_ocupacao,grupo_ocupacao,quantidade_vinculos
144,Assistente Administrativo,Trabalhadores De Serviços Administrativos,902303
1599,Professor De Nível Médio No Ensino Fundamental,Profissionais De Ensino,610685
1601,Professor De Nível Superior Do Ensino Fundamen...,Profissionais De Ensino,463232
544,Dirigente Do Serviço Público Municipal,"Membros Superiores Do Poder Público, Dirigente...",342934
542,Dirigente Do Serviço Público Estadual E Distrital,"Membros Superiores Do Poder Público, Dirigente...",322022
184,Auxiliar De Escritório,Trabalhadores De Serviços Administrativos,311322
1888,Trabalhador De Serviços De Limpeza E Conservaç...,"Trabalhadores Dos Serviços, Vendedores Do Comé...",288519
1515,Professor Da Educação De Jovens E Adultos Do ...,Profissionais De Ensino,284109
1933,Técnico De Enfermagem,"Profissionais De Medicina, Enfermagem E Serviç...",264014
26,Agente Comunitário De Saúde,"Profissionais De Medicina, Enfermagem E Serviç...",243167


In [22]:
top10 = top10.rename(columns={'quantidade_vinculos':'vinculo_profissoes'})

In [36]:
top10_merge = top10.merge(df_agrupado, how='left', on='grupo_ocupacao') #pegando total por grupo
top10_merge

,nome_ocupacao,grupo_ocupacao,vinculo_profissoes,quantidade_vinculos,proporcao
0,Assistente Administrativo,Trabalhadores De Serviços Administrativos,902303,1862521,18.687994
1,Professor De Nível Médio No Ensino Fundamental,Profissionais De Ensino,610685,2670616,26.796184
2,Professor De Nível Superior Do Ensino Fundamen...,Profissionais De Ensino,463232,2670616,26.796184
3,Dirigente Do Serviço Público Municipal,"Membros Superiores Do Poder Público, Dirigente...",342934,915395,9.184807
4,Dirigente Do Serviço Público Estadual E Distrital,"Membros Superiores Do Poder Público, Dirigente...",322022,915395,9.184807
5,Auxiliar De Escritório,Trabalhadores De Serviços Administrativos,311322,1862521,18.687994
6,Trabalhador De Serviços De Limpeza E Conservaç...,"Trabalhadores Dos Serviços, Vendedores Do Comé...",288519,917772,9.208657
7,Professor Da Educação De Jovens E Adultos Do ...,Profissionais De Ensino,284109,2670616,26.796184
8,Técnico De Enfermagem,"Profissionais De Medicina, Enfermagem E Serviç...",264014,1425713,14.30519
9,Agente Comunitário De Saúde,"Profissionais De Medicina, Enfermagem E Serviç...",243167,1425713,14.30519


In [37]:
top10_merge = top10_merge.rename(columns={'proporcao':'proporcao_grupo'})
top10_merge

,nome_ocupacao,grupo_ocupacao,vinculo_profissoes,quantidade_vinculos,proporcao_grupo
0,Assistente Administrativo,Trabalhadores De Serviços Administrativos,902303,1862521,18.687994
1,Professor De Nível Médio No Ensino Fundamental,Profissionais De Ensino,610685,2670616,26.796184
2,Professor De Nível Superior Do Ensino Fundamen...,Profissionais De Ensino,463232,2670616,26.796184
3,Dirigente Do Serviço Público Municipal,"Membros Superiores Do Poder Público, Dirigente...",342934,915395,9.184807
4,Dirigente Do Serviço Público Estadual E Distrital,"Membros Superiores Do Poder Público, Dirigente...",322022,915395,9.184807
5,Auxiliar De Escritório,Trabalhadores De Serviços Administrativos,311322,1862521,18.687994
6,Trabalhador De Serviços De Limpeza E Conservaç...,"Trabalhadores Dos Serviços, Vendedores Do Comé...",288519,917772,9.208657
7,Professor Da Educação De Jovens E Adultos Do ...,Profissionais De Ensino,284109,2670616,26.796184
8,Técnico De Enfermagem,"Profissionais De Medicina, Enfermagem E Serviç...",264014,1425713,14.30519
9,Agente Comunitário De Saúde,"Profissionais De Medicina, Enfermagem E Serviç...",243167,1425713,14.30519


In [44]:
total_vinculos22 = df_agrupado['quantidade_vinculos'].sum()
total_vinculos22

9966404

In [39]:
top10_merge['proporcao_prof_vinculos_totais2022'] = (top10_merge['vinculo_profissoes']/total_vinculos22)*100 #em relação ao total

In [40]:
top10_merge['proporcao_profissao'] = (top10_merge['vinculo_profissoes'] / top10_merge['quantidade_vinculos'])*100
#proporção da profissao em relação ao grupo 


In [41]:
top10_merge

,nome_ocupacao,grupo_ocupacao,vinculo_profissoes,quantidade_vinculos,proporcao_grupo,proporcao_prof_vinculos_totais2022,proporcao_profissao
0,Assistente Administrativo,Trabalhadores De Serviços Administrativos,902303,1862521,18.687994,9.053446,48.445252
1,Professor De Nível Médio No Ensino Fundamental,Profissionais De Ensino,610685,2670616,26.796184,6.127436,22.866822
2,Professor De Nível Superior Do Ensino Fundamen...,Profissionais De Ensino,463232,2670616,26.796184,4.647935,17.345511
3,Dirigente Do Serviço Público Municipal,"Membros Superiores Do Poder Público, Dirigente...",342934,915395,9.184807,3.4409,37.462953
4,Dirigente Do Serviço Público Estadual E Distrital,"Membros Superiores Do Poder Público, Dirigente...",322022,915395,9.184807,3.231075,35.178475
5,Auxiliar De Escritório,Trabalhadores De Serviços Administrativos,311322,1862521,18.687994,3.123714,16.715087
6,Trabalhador De Serviços De Limpeza E Conservaç...,"Trabalhadores Dos Serviços, Vendedores Do Comé...",288519,917772,9.208657,2.894916,31.436893
7,Professor Da Educação De Jovens E Adultos Do ...,Profissionais De Ensino,284109,2670616,26.796184,2.850667,10.638332
8,Técnico De Enfermagem,"Profissionais De Medicina, Enfermagem E Serviç...",264014,1425713,14.30519,2.64904,18.518033
9,Agente Comunitário De Saúde,"Profissionais De Medicina, Enfermagem E Serviç...",243167,1425713,14.30519,2.439867,17.055817


In [43]:
top10_merge.to_excel('2022 top 10 profissoes + %.xlsx')

In [42]:
os.listdir()

['errado.gsheet',
 'total por grupo 2022.xlsx',
 'dados comparativos - CBO 2003 2022 .xlsx',
 'total por grupo 2003.xlsx',
 'top10 profissoes dos Trabalhadores De Serviços Administrativos 2003.csv',
 '2003 top 10 profissoes + %.xlsx']

In [45]:
df_agrupado.columns

Index(['grupo_ocupacao', 'quantidade_vinculos', 'proporcao'], dtype='object')

# maior qtd de vinculos 2022 por profissao

In [48]:
df.columns

Index(['ano', 'codigo', 'nome_ocupacao', 'grupo_ocupacao', 'esfera', 'genero',
       'quantidade_vinculos'],
      dtype='object')

In [56]:
df_agp = df.groupby(['nome_ocupacao', 'grupo_ocupacao']).agg({'quantidade_vinculos': 'sum'}).reset_index()
df_agp.sort_values(ascending=False, by= 'quantidade_vinculos').head(10)

,nome_ocupacao,grupo_ocupacao,quantidade_vinculos
144,Assistente Administrativo,Trabalhadores De Serviços Administrativos,902303
1599,Professor De Nível Médio No Ensino Fundamental,Profissionais De Ensino,610685
1601,Professor De Nível Superior Do Ensino Fundamen...,Profissionais De Ensino,463232
544,Dirigente Do Serviço Público Municipal,"Membros Superiores Do Poder Público, Dirigente...",342934
542,Dirigente Do Serviço Público Estadual E Distrital,"Membros Superiores Do Poder Público, Dirigente...",322022
184,Auxiliar De Escritório,Trabalhadores De Serviços Administrativos,311322
1888,Trabalhador De Serviços De Limpeza E Conservaç...,"Trabalhadores Dos Serviços, Vendedores Do Comé...",288519
1515,Professor Da Educação De Jovens E Adultos Do ...,Profissionais De Ensino,284109
1933,Técnico De Enfermagem,"Profissionais De Medicina, Enfermagem E Serviç...",264014
26,Agente Comunitário De Saúde,"Profissionais De Medicina, Enfermagem E Serviç...",243167


In [59]:
df_agp22 = df_agp.sort_values(ascending=False, by= 'quantidade_vinculos')
df_agp22

,nome_ocupacao,grupo_ocupacao,quantidade_vinculos
144,Assistente Administrativo,Trabalhadores De Serviços Administrativos,902303
1599,Professor De Nível Médio No Ensino Fundamental,Profissionais De Ensino,610685
1601,Professor De Nível Superior Do Ensino Fundamen...,Profissionais De Ensino,463232
544,Dirigente Do Serviço Público Municipal,"Membros Superiores Do Poder Público, Dirigente...",342934
542,Dirigente Do Serviço Público Estadual E Distrital,"Membros Superiores Do Poder Público, Dirigente...",322022
...,...,...,...
1737,Supervisor Da Indústria De Fumo,Trabalhadores Da Produção De Bens E Serviços I...,1
758,Forneiro No Recozimento De Vidro,Trabalhadores Da Produção De Bens E Serviços I...,1
1759,Supervisor De Embalagem E Etiquetagem,Trabalhadores Da Produção De Bens E Serviços I...,1
978,Matemático Aplicado,Profissionais De Ciência De Ensino Superior,1


In [57]:
os.listdir()

['errado.gsheet',
 'total por grupo 2022.xlsx',
 'dados comparativos - CBO 2003 2022 .xlsx',
 'total por grupo 2003.xlsx',
 'top10 profissoes dos Trabalhadores De Serviços Administrativos 2003.csv',
 '2003 top 10 profissoes + %.xlsx',
 '2022 top 10 profissoes + %.xlsx',
 'quantidae vinculo 2003.xlsx']

In [58]:
df_agp3 = pd.read_excel('quantidae vinculo 2003.xlsx')
df_agp3

,nome_ocupacao,grupo_ocupacao,quantidade_vinculos
0,Assistente Administrativo,Trabalhadores De Serviços Administrativos,751741
1,Professor De Nível Médio No Ensino Fundamental,Profissionais De Ensino,616824
2,Auxiliar De Escritório,Trabalhadores De Serviços Administrativos,598009
3,Professor De Nível Superior Do Ensino Fundamen...,Profissionais De Ensino,345480
4,Dirigente Do Serviço Público Estadual E Distrital,"Membros Superiores Do Poder Público, Dirigente...",200756
...,...,...,...
2072,Estatístico Teórico,Profissionais De Ciência De Ensino Superior,1
2073,Eletromecânico De Manutenção De Portas Automát...,Trabalhadores Em Serviços De Reparação E Manut...,1
2074,"Operador De Bobinadeira De Tiras A Quente, No ...",Trabalhadores Da Produção De Bens E Serviços I...,1
2075,Decorador De Cerâmica,Trabalhadores Da Produção De Bens E Serviços I...,1


In [64]:
df_agp3.columns

Index(['nome_ocupacao', 'grupo_ocupacao', 'quantidade_vinculos'], dtype='object')

In [65]:
df_agp22.columns

Index(['nome_ocupacao', 'grupo_ocupacao', 'quantidade_vinculos'], dtype='object')

In [73]:
comparacao = df_agp3.merge(df_agp22, on='nome_ocupacao', how='left', suffixes=('03','22'))
comparacao

,nome_ocupacao,grupo_ocupacao03,quantidade_vinculos03,grupo_ocupacao22,quantidade_vinculos22
0,Assistente Administrativo,Trabalhadores De Serviços Administrativos,751741,Trabalhadores De Serviços Administrativos,902303
1,Professor De Nível Médio No Ensino Fundamental,Profissionais De Ensino,616824,Profissionais De Ensino,610685
2,Auxiliar De Escritório,Trabalhadores De Serviços Administrativos,598009,Trabalhadores De Serviços Administrativos,311322
3,Professor De Nível Superior Do Ensino Fundamen...,Profissionais De Ensino,345480,Profissionais De Ensino,463232
4,Dirigente Do Serviço Público Estadual E Distrital,"Membros Superiores Do Poder Público, Dirigente...",200756,"Membros Superiores Do Poder Público, Dirigente...",322022
...,...,...,...,...,...
2072,Estatístico Teórico,Profissionais De Ciência De Ensino Superior,1,Profissionais De Ciência De Ensino Superior,1
2073,Eletromecânico De Manutenção De Portas Automát...,Trabalhadores Em Serviços De Reparação E Manut...,1,Trabalhadores Em Serviços De Reparação E Manut...,5
2074,"Operador De Bobinadeira De Tiras A Quente, No ...",Trabalhadores Da Produção De Bens E Serviços I...,1,NaN,<NA>
2075,Decorador De Cerâmica,Trabalhadores Da Produção De Bens E Serviços I...,1,Trabalhadores Da Produção De Bens E Serviços I...,3


In [74]:
comparacao[comparacao['nome_ocupacao']=='Agente De Saúde Pública']

,nome_ocupacao,grupo_ocupacao03,quantidade_vinculos03,grupo_ocupacao22,quantidade_vinculos22
22,Agente Comunitário De Saúde,"Profissionais De Medicina, Enfermagem E Serviç...",57032,"Profissionais De Medicina, Enfermagem E Serviç...",243167


In [75]:
comparacao['diferenca'] = comparacao['quantidade_vinculos22'] - comparacao['quantidade_vinculos03']


In [87]:
posi = comparacao.sort_values(ascending=False, by='diferenca').head(10)
posi

,nome_ocupacao,grupo_ocupacao03,quantidade_vinculos03,grupo_ocupacao22,quantidade_vinculos22,diferenca
46,Técnico De Enfermagem,"Profissionais De Medicina, Enfermagem E Serviç...",24281,"Profissionais De Medicina, Enfermagem E Serviç...",264014,239733
22,Agente Comunitário De Saúde,"Profissionais De Medicina, Enfermagem E Serviç...",57032,"Profissionais De Medicina, Enfermagem E Serviç...",243167,186135
9,Dirigente Do Serviço Público Municipal,"Membros Superiores Do Poder Público, Dirigente...",163410,"Membros Superiores Do Poder Público, Dirigente...",342934,179524
0,Assistente Administrativo,Trabalhadores De Serviços Administrativos,751741,Trabalhadores De Serviços Administrativos,902303,150562
4,Dirigente Do Serviço Público Estadual E Distrital,"Membros Superiores Do Poder Público, Dirigente...",200756,"Membros Superiores Do Poder Público, Dirigente...",322022,121266
26,Professor De Nível Médio Na Educação Infantil,Profissionais De Ensino,48321,Profissionais De Ensino,168549,120228
3,Professor De Nível Superior Do Ensino Fundamen...,Profissionais De Ensino,345480,Profissionais De Ensino,463232,117752
29,Enfermeiro,"Profissionais De Medicina, Enfermagem E Serviç...",42058,"Profissionais De Medicina, Enfermagem E Serviç...",156699,114641
6,Trabalhador De Serviços De Limpeza E Conservaç...,"Trabalhadores Dos Serviços, Vendedores Do Comé...",192410,"Trabalhadores Dos Serviços, Vendedores Do Comé...",288519,96109
103,Professor De Ensino Superior Na Área De Didática,Profissionais De Ensino,8740,Profissionais De Ensino,104046,95306
